In [1]:
from pyscf import scf,gto
from alch_deriv import alch_deriv
import pyscf
import numpy as np
import inspect
from FcMole import FcM
from functools import reduce
import matplotlib.pyplot as plt
from pyscf.scf import cphf
from pyscf.grad import rhf as grhf  #### very important
from pyscf.hessian import rhf as hrhf # without those two mf.Gradients() and mf.Hessian() don't work
def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV

In [2]:
mol=gto.M(atom="H 0 0 0; Li 0 0 1.8",unit="Bohr",basis="STO-3G")
mf=scf.RHF(mol)
mf.scf(dm0=mf.init_guess_by_atom())
P=mf.make_rdm1()
hcore=mf.get_hcore()
h=mf.Hessian()
h.kernel()
pass

converged SCF energy = -7.74106569060783


/home/giorgiod/miniconda3/lib/python3.7/site-packages/pyscf/lib/chkfile.py:71: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  return val.value


In [102]:
# mol with fract. charges
fcs=[.1,-.1]
fmol=FcM(fcs=fcs,atom="H 0 0 0; Li 0 0 1.8",unit="Bohr",basis="STO-3G")
fmf=scf.RHF(fmol)
fmf.scf(dm0=fmf.init_guess_by_atom())
fP=fmf.make_rdm1()
fhcore=fmf.get_hcore()
fh=fmf.Hessian()
fh.kernel()
pass

converged SCF energy = -7.29896716736342


In [103]:
# mol with streched bond  
mol1=gto.M(atom="H 0 0 0; Li 0 0 1.801",unit="Bohr",basis="STO-3G")
mf1=scf.RHF(mol1)
mf1.scf()
hcore1=mf1.get_hcore()
P1=mf1.make_rdm1()
h1=mf1.Hessian()
h1.kernel()
dP_fd=P1-P

converged SCF energy = -7.7413864568394


In [104]:
mo_energy = mf.mo_energy
mo_occ = mf.mo_occ
mo_coeff = mf.mo_coeff
C=mo_coeff
atmlst = range(mol.natm)
mocc = mo_coeff[:,mo_occ>0]

In [105]:
fmo_energy = fmf.mo_energy
fmo_occ = fmf.mo_occ
fmo_coeff = fmf.mo_coeff
fC=mo_coeff
fatmlst = range(fmol.natm)
fmocc = fmo_coeff[:,mo_occ>0]
fh1ao = fh.make_h1(fmo_coeff, fmo_occ)
fmo1_a0,fmo1_a1=fh.solve_mo1(fmo_energy,fmo_coeff,fmo_occ,fh1ao)[0]

In [106]:
h1ao = h.make_h1(mo_coeff,mo_occ)
#h1ao[0][:]-=DeltaV(mol,[.01,-.01])
#h1ao[1][:]-=DeltaV(mol,[.01,-.01])
h1ao[0].shape

(3, 6, 6)

In [107]:
mo1_a0,mo1_a1=h.solve_mo1(fmo_energy,fmo_coeff,fmo_occ,h1ao)[0]

In [108]:
dm1_0 = np.einsum('ypi,qi->ypq', mo1_a0, fmocc) #shape=(nocc,nocc)
dm1_0[2]=(dm1_0[2]+dm1_0[2].T)*2
print(np.round(dm1_0[2],decimals=4))

[[ 0.0309  0.0363 -0.1188 -0.     -0.     -0.1399]
 [ 0.0363  0.0509 -0.1883 -0.      0.      0.2928]
 [-0.1188 -0.1883 -0.1065 -0.     -0.     -0.0034]
 [-0.     -0.     -0.      0.      0.      0.    ]
 [-0.      0.     -0.      0.      0.      0.    ]
 [-0.1399  0.2928 -0.0034  0.      0.      0.1914]]


In [109]:
fdm1_0 = np.einsum('ypi,qi->ypq', fmo1_a0, fmocc)
fdm1_0[2]=(fdm1_0[2]+fdm1_0[2].T)*2
print(np.round(fdm1_0[2],decimals=4))

[[-0.0405  0.0091 -0.093  -0.     -0.     -0.1355]
 [ 0.0091  0.0624 -0.1816 -0.      0.      0.2657]
 [-0.093  -0.1816 -0.0708 -0.     -0.     -0.0317]
 [-0.     -0.     -0.      0.      0.      0.    ]
 [-0.      0.     -0.      0.      0.      0.    ]
 [-0.1355  0.2657 -0.0317  0.      0.      0.2028]]


In [110]:
aoslices = mol.aoslice_by_atom()
faoslices = fmol.aoslice_by_atom()
aoslices==faoslices

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [111]:
"""def make_h1( mo_coeff, mo_occ, chkfile=None, atmlst=None, verbose=None):
  #  mol = hessobj.mol
    if atmlst is None:
        atmlst = range(mol.natm)
    nao, nmo = mo_coeff.shape
    mocc = mo_coeff[:,mo_occ>0]
    dm0 = numpy.dot(mocc, mocc.T) * 2
    hcore_deriv = hessobj.base.nuc_grad_method().hcore_generator(mol)   #dH/dx need to adj for new charges
    aoslices = mol.aoslice_by_atom()
    h1ao = [None] * mol.natm
    for i0, ia in enumerate(atmlst):
        shl0, shl1, p0, p1 = aoslices[ia]
        shls_slice = (shl0, shl1) + (0, mol.nbas)*3
        vj1, vj2, vk1, vk2 = _get_jk(mol, 'int2e_ip1', 3, 's2kl',
                                     ['ji->s2kl', -dm0[:,p0:p1],  # vj1
                                      'lk->s1ij', -dm0         ,  # vj2
                                      'li->s1kj', -dm0[:,p0:p1],  # vk1
                                      'jk->s1il', -dm0         ], # vk2
                                     shls_slice=shls_slice)
        vhf = vj1 - vk1*.5
        vhf[:,p0:p1] += vj2 - vk2*.5
        h1 = vhf + vhf.transpose(0,2,1)
        h1 += hcore_deriv(ia)
"""

"def make_h1( mo_coeff, mo_occ, chkfile=None, atmlst=None, verbose=None):\n  #  mol = hessobj.mol\n    if atmlst is None:\n        atmlst = range(mol.natm)\n    nao, nmo = mo_coeff.shape\n    mocc = mo_coeff[:,mo_occ>0]\n    dm0 = numpy.dot(mocc, mocc.T) * 2\n    hcore_deriv = hessobj.base.nuc_grad_method().hcore_generator(mol)   #dH/dx need to adj for new charges\n    aoslices = mol.aoslice_by_atom()\n    h1ao = [None] * mol.natm\n    for i0, ia in enumerate(atmlst):\n        shl0, shl1, p0, p1 = aoslices[ia]\n        shls_slice = (shl0, shl1) + (0, mol.nbas)*3\n        vj1, vj2, vk1, vk2 = _get_jk(mol, 'int2e_ip1', 3, 's2kl',\n                                     ['ji->s2kl', -dm0[:,p0:p1],  # vj1\n                                      'lk->s1ij', -dm0         ,  # vj2\n                                      'li->s1kj', -dm0[:,p0:p1],  # vk1\n                                      'jk->s1il', -dm0         ], # vk2\n                                     shls_slice=shls_slice)\n        vh

In [112]:
"""def hcore_generator(mf, mol=None):    # on gradient module 
    aoslices = mol.aoslice_by_atom()
    h1 = mf.get_hcore(mol)   # gradients method : -\nabla_r (T+V_en)    is different between mol and fmol
    def hcore_deriv(atm_id):
        shl0, shl1, p0, p1 = aoslices[atm_id]
        with mol.with_rinv_at_nucleus(atm_id):
            vrinv = mol.intor('int1e_iprinv', comp=3) # <\nabla_R 1/|R-r|>    this is equal 
            vrinv *= -mol.atom_charge(atm_id)        # atom charges should be updated 
        vrinv[:,p0:p1] += h1[:,p0:p1]
        return vrinv + vrinv.transpose(0,2,1)
    return hcore_deriv
"""
pass

In [113]:
"""def get_hcore(mol):    # on gradient module 
    '''Part of the nuclear gradients of core Hamiltonian'''
    h = mol.intor('int1e_ipkin', comp=3)
    h+= mol.intor('int1e_ipnuc', comp=3)
    return -h"""
pass

In [114]:
(np.allclose(grhf.get_hcore(fmol),grhf.get_hcore(mol))) #false

False

In [115]:
np.allclose(-mol.intor('int1e_ipkin', comp=3)-mol.intor('int1e_ipnuc', comp=3),grhf.get_hcore(mol)) #true
np.allclose(mol.intor('int1e_ipkin', comp=3),fmol.intor('int1e_ipkin', comp=3)) #true
np.allclose(mol.intor('int1e_ipnuc', comp=3),fmol.intor('int1e_ipnuc', comp=3)) #False

False

In [116]:
print(mol.aoslice_by_atom())
for atm_id in [0,1]:
    mol.with_rinv_at_nucleus(atm_id) 
    fmol.with_rinv_at_nucleus(atm_id)
    print(mol.intor('int1e_iprinv', comp=3).shape )
    print(np.allclose(mol.intor('int1e_iprinv', comp=3),fmol.intor('int1e_iprinv', comp=3)))

[[0 1 0 1]
 [1 4 1 6]]
(3, 6, 6)
False
(3, 6, 6)
False


the differnces are in " mol.intor('int1e_ipnuc', comp=3) " 

In [117]:
np.allclose(grhf.get_hcore(mol),grhf.get_hcore(fmol))
grhf.get_hcore(mol).shape

(3, 6, 6)

The value of $ <\chi | \nabla T|\mu >$ calculated with "mol.intor('int1e_iprinv', comp=3)" is identical for mol and fmol.
The only thing to do is to update ,mol.intor('int1e_ipnuc', comp=3)

In [118]:
#print(inspect.getsource(mol.intor))
#print(inspect.getsource(mol._add_suffix))

In [119]:
#environment 
NUC_FRAC_CHARGE=gto.mole.NUC_FRAC_CHARGE
NUC_MOD_OF=gto.mole.NUC_MOD_OF
PTR_FRAC_CHARGE=gto.mole.PTR_FRAC_CHARGE
denv=mol._env.copy()
datm=mol._atm.copy()
datm[:,NUC_MOD_OF] = NUC_FRAC_CHARGE
for i in range (mol.natm):
    denv[datm[i,PTR_FRAC_CHARGE]]=fcs[i]
dH1=-gto.moleintor.getints('int1e_ipnuc_sph',datm,mol._bas,denv, None,3,0,'s1')
comp2=np.zeros((2,3,6,6))
for atm_id in [0,1]:
    shl0, shl1, p0, p1 = mol.aoslice_by_atom()[atm_id]
    with mol.with_rinv_at_nucleus(atm_id):
        vrinv = -mol.intor('int1e_iprinv', comp=3)
    vrinv*=fcs[atm_id]
    vrinv[:,p0:p1] += dH1[:,p0:p1] 
    vrinv += vrinv.transpose(0,2,1)
    comp2[atm_id]=vrinv

In [120]:
np.allclose(-dH1,grhf.get_hcore(mol)-grhf.get_hcore(fmol))

True

In [121]:
np.allclose(fh.make_h1(fmo_coeff,fmo_occ),h.make_h1(fmo_coeff,fmo_occ)+comp2)

True

In [122]:
h.base.nuc_grad_method().hcore_generator(fmol)(1)
np.allclose(fh.base.nuc_grad_method().hcore_generator(fmol)(1),h.base.nuc_grad_method().hcore_generator(mol)(1)+comp2[1])#true
np.allclose(fh.base.nuc_grad_method().hcore_generator(fmol)(0),h.base.nuc_grad_method().hcore_generator(mol)(0)+comp2[0])

True

In [123]:
np.allclose(fh.make_h1(fmo_coeff,fmo_occ),h.make_h1(fmo_coeff,fmo_occ)+comp2)

True

Here I can finish the work, adding to make_h1 the derivative "comp2"

In [168]:
pmo1_a0,pmo1_a1=h.solve_mo1(fmo_energy,fmo_coeff,fmo_occ,h1ao+comp2)[0]

In [169]:
pdm1_0 = np.einsum('ypi,qi->ypq', pmo1_a0, fmocc) #shape=(nocc,nocc)
pdm1_0[2]=(pdm1_0[2]+pdm1_0[2].T)*2
print(np.round(pdm1_0[2]-dm1_0[2],decimals=4))
print(np.round(fdm1_0[2]-dm1_0[2],decimals=4))


[[-0.0718 -0.0236  0.0181  0.      0.     -0.0017]
 [-0.0236  0.0092  0.0103 -0.     -0.     -0.0245]
 [ 0.0181  0.0103  0.029   0.      0.     -0.0266]
 [ 0.     -0.      0.      0.      0.     -0.    ]
 [ 0.     -0.      0.      0.      0.     -0.    ]
 [-0.0017 -0.0245 -0.0266 -0.     -0.      0.019 ]]
[[-0.0714 -0.0273  0.0257  0.      0.      0.0044]
 [-0.0273  0.0115  0.0067 -0.     -0.     -0.0271]
 [ 0.0257  0.0067  0.0357  0.      0.     -0.0283]
 [ 0.     -0.      0.      0.      0.     -0.    ]
 [ 0.     -0.      0.      0.      0.     -0.    ]
 [ 0.0044 -0.0271 -0.0283 -0.     -0.      0.0114]]


In [126]:
U,dP,pe1= alch_deriv(mf,dL=[[0,1],[.1,-.1]])

In [127]:
pmo_coeff=mo_coeff+mo_coeff@U
pmocc=pmo_coeff[:,mo_occ>0]

In [146]:
pmo_energy=np.concatenate((mo_energy[:4]+pe1.flatten(),mo_energy[4:]))
pmo_energy,fmo_energy,pe1

(array([-2.15977917, -0.35857268,  0.03274593,  0.14237875,  0.15659232,
         0.60740557]),
 array([-2.16300477, -0.33217758,  0.09389118,  0.17209773,  0.17209773,
         0.57176033]),
 array([[[ 0.22125914, -0.04279833],
         [-0.04279833, -0.01421357]]]))

In [176]:
pmo1_a0,pmo1_a1=h.solve_mo1(mo_energy,pmo_coeff,mo_occ,h1ao+comp2)[0]

In [179]:
pdm1_0 = np.einsum('ypi,qi->ypq', pmo1_a0, pmocc) #shape=(nocc,nocc)
pdm1_0[2]=(pdm1_0[2]+pdm1_0[2].T)*2
print(np.round(pdm1_0[2],decimals=3))
print(np.round(fdm1_0[2],decimals=3))
print(np.round(dm1_0[2],decimals=3))

[[ 0.016 -0.105 -0.138 -0.     0.    -0.168]
 [-0.105  0.076 -0.074  0.    -0.     0.186]
 [-0.138 -0.074 -0.122 -0.     0.     0.007]
 [-0.     0.    -0.     0.    -0.     0.   ]
 [ 0.    -0.     0.    -0.     0.    -0.   ]
 [-0.168  0.186  0.007  0.    -0.     0.205]]
[[-0.041  0.009 -0.093 -0.    -0.    -0.135]
 [ 0.009  0.062 -0.182 -0.     0.     0.266]
 [-0.093 -0.182 -0.071 -0.    -0.    -0.032]
 [-0.    -0.    -0.     0.     0.     0.   ]
 [-0.     0.    -0.     0.     0.     0.   ]
 [-0.135  0.266 -0.032  0.     0.     0.203]]
[[ 0.031  0.036 -0.119 -0.    -0.    -0.14 ]
 [ 0.036  0.051 -0.188 -0.     0.     0.293]
 [-0.119 -0.188 -0.106 -0.    -0.    -0.003]
 [-0.    -0.    -0.     0.     0.     0.   ]
 [-0.     0.    -0.     0.     0.     0.   ]
 [-0.14   0.293 -0.003  0.     0.     0.191]]


In [153]:
"""
def solve_mo1(mf, mo_energy, mo_coeff, mo_occ, h1ao_or_chkfile,
              fx=None, atmlst=None, max_memory=4000, verbose=None):
    '''Solve the first order equation

    Kwargs:
        fx : function(dm_mo) => v1_mo
            A function to generate the induced potential.
            See also the function gen_vind.
    '''
    mol = mf.mol
    if atmlst is None: atmlst = range(mol.natm)
    nao, nmo = mo_coeff.shape
    mocc = mo_coeff[:,mo_occ>0]
    nocc = mocc.shape[1]

    if fx is None:
        fx = gen_vind(mf, mo_coeff, mo_occ)
    s1a = -mol.intor('int1e_ipovlp', comp=3)

    def _ao2mo(mat):
        return numpy.asarray([reduce(numpy.dot, (mo_coeff.T, x, mocc)) for x in mat])

    mo1s = [None] * mol.natm
    e1s = [None] * mol.natm
    aoslices = mol.aoslice_by_atom()
    for ia0, ia1 in lib.prange(0, len(atmlst), blksize):
        s1vo = []
        h1vo = []
        for i0 in range(ia0, ia1):
            ia = atmlst[i0]
            shl0, shl1, p0, p1 = aoslices[ia]
            s1ao = numpy.zeros((3,nao,nao))
            s1ao[:,p0:p1] += s1a[:,p0:p1]
            s1ao[:,:,p0:p1] += s1a[:,p0:p1].transpose(0,2,1)
            s1vo.append(_ao2mo(s1ao))
            if isinstance(h1ao_or_chkfile, str):
                key = 'scf_f1ao/%d' % ia
                h1ao = lib.chkfile.load(h1ao_or_chkfile, key)
            else:
                h1ao = h1ao_or_chkfile[ia]
            h1vo.append(_ao2mo(h1ao))

        h1vo = numpy.vstack(h1vo)
        s1vo = numpy.vstack(s1vo)
        mo1, e1 = cphf.solve(fx, mo_energy, mo_occ, h1vo, s1vo)
        mo1 = numpy.einsum('pq,xqi->xpi', mo_coeff, mo1).reshape(-1,3,nao,nocc)    ###### imortant C@U
        e1 = e1.reshape(-1,3,nocc,nocc) 

        for k in range(ia1-ia0):
            ia = atmlst[k+ia0]
            if isinstance(h1ao_or_chkfile, str):
                key = 'scf_mo1/%d' % ia
                lib.chkfile.save(h1ao_or_chkfile, key, mo1[k])
            else:
                mo1s[ia] = mo1[k]
            e1s[ia] = e1[k].reshape(3,nocc,nocc)
        mo1 = e1 = None

    if isinstance(h1ao_or_chkfile, str):
        return h1ao_or_chkfile, e1s
    else:
        return mo1s, e1s
"""
pass

In [ ]:
"""from pyscf import lib
import numpy
h1ao_or_chkfile=h.make_h1(mo_coeff,mo_occ)
nao, nmo = mo_coeff.shape
mocc = mo_coeff[:,mo_occ>0]
nocc = mocc.shape[1]

fx = hrhf.gen_vind(mf, mo_coeff, mo_occ)
s1a = -mol.intor('int1e_ipovlp', comp=3)
blksize = max(2, int(4000*1e6/8 / (nmo*nocc*3*6)))
def _ao2mo(mat):
    return numpy.asarray([reduce(numpy.dot, (mo_coeff.T, x, mocc)) for x in mat])

mo1s = [None] * mol.natm
e1s = [None] * mol.natm
aoslices = mol.aoslice_by_atom()
for ia0, ia1 in lib.prange(0, len(atmlst), blksize):
    s1vo = []
    h1vo = []
    for i0 in range(ia0, ia1):
        ia = atmlst[i0]
        shl0, shl1, p0, p1 = aoslices[ia]
        s1ao = numpy.zeros((3,nao,nao))
        s1ao[:,p0:p1] += s1a[:,p0:p1]
        s1ao[:,:,p0:p1] += s1a[:,p0:p1].transpose(0,2,1)
        s1vo.append(_ao2mo(s1ao))
        if isinstance(h1ao_or_chkfile, str):
            key = 'scf_f1ao/%d' % ia
            h1ao = lib.chkfile.load(h1ao_or_chkfile, key)
        else:
            h1ao = h1ao_or_chkfile[ia]
        h1vo.append(_ao2mo(h1ao))
    h1vo = numpy.vstack(h1vo)
    s1vo = numpy.vstack(s1vo)
    mo1, e1 = cphf.solve(fx, mo_energy, mo_occ, h1vo, s1vo)
    """
pass